In [ ]:
# https://doi.org/10.1016/j.ijforecast.2008.07.004
# https://github.com/robjhyndman/reconciliation_review_talk/blob/main/10years_reconciliation.pdf
# https://otexts.com/fpp3/single-level.html


import pandas as pd

from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.transformations.hierarchical.aggregate import Aggregator
from sktime.transformations.hierarchical.reconcile import Reconciler
from sktime.utils._testing.hierarchical import _bottom_hier_datagen

# get all the methods

METHOD_LIST = Reconciler.METHOD_LIST

In [ ]:
agg = Aggregator(flatten_single_levels=True)

X = _bottom_hier_datagen(
    no_bottom_nodes=3,
    no_levels=1,
)
# add aggregate levels
X = agg.fit_transform(X)

# forecast all levels
fh = ForecastingHorizon([1, 2], is_relative=True)
forecaster = ExponentialSmoothing(trend="add", seasonal="additive", sp=12)
prds = forecaster.fit(X).predict(fh)

In [ ]:
prds

In [ ]:
prds["bu"] = Aggregator().fit_transform(prds)
prds

In [ ]:
tmp = agg.inverse_transform(prds[["passengers"]])
tmp["props"] = tmp / tmp.groupby(level=-1).transform(sum)
tmp

In [ ]:
tot = prds.loc[
    prds.index.get_level_values(level=-2).isin(["__total"]), ["passengers"]
].copy()
tot.columns = ["total"]
tot

In [ ]:
tmp_drop = tmp.droplevel(0)
tmp_noind = pd.concat(
    [tmp.droplevel(0), tot.droplevel(0).reindex(tmp_drop.index)], axis=1
)
tmp_noind.index = tmp.index
tmp = tmp_noind
tmp

In [ ]:
tmp["recon"] = tmp["props"] * tmp["total"]
tmp

In [ ]:
prds["td_fcst"] = agg.fit_transform(tmp[["recon"]])
prds